In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
import DifferentialEquations as DE
using CairoMakie
using GLMakie
include("multiharmonic_balance.jl");

# Harmonic Balance using multiple harmonics

## Use the SymPy method to start a potential Harmonic Balance

$$
u = \sum_{k=1}^H A_k \cos(k\omega t) + B_k \sin(k\omega t)
$$

Define the variable parameters for the wave equation

In [ ]:
gamma = 0;
omega = 3;
gamma3 = 0;
g0 = 9.80665; # m / s^2
height = 5; # m
harmonics = 1; # number of harmonics

Define the constants specific to the discretizations

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
Nt = 5
N = 100;
order = 2;
stepx = (xright-xleft)/N;

In [ ]:
# Define symbolics
Model.@parameters x, t;

Dx = Model.Differential(x);
Dt = Model.Differential(t);

In [ ]:
u0 = ones((N - 1) * harmonics * 2);

In [ ]:
vars, var_exprs, u = create_ansatz(x, t, omega, harmonics);
bcs = create_bcs(vars, [xleft, xright], 0);

F = 50 * exp(-40*(x^2))*sin(omega*t)

y = Dt(Dt(u)) - 9*Dx(Dx(u)) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u) - F;

Model.@variables x, t;
expanded = expand_trig(y, x, t, omega);

eqs = make_residual(expanded, harmonics, omega, t);

In [ ]:
var_exprs

In [ ]:
solution_coeffs = solve_harmonicbalance(eqs, harmonics, bcs, [xleft, xright], stepx, x, var_exprs, u0);

In [ ]:
solution_coeffs